In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
ahu = pd.read_csv("/content/drive/MyDrive/ime_project/data/porest_area_ahu_automatic.csv")
chiller = pd.read_csv("/content/drive/MyDrive/ime_project/data/porest_area_chiller_automatic.csv")

column_e = ['no', 'id', 'seq', 'type', 'val', 'rv', 'ri', 'sv', 'si', 'tv', 'ti', 'fire', 'temperature', 'humidity', 'cdate']
e = pd.read_csv('/content/drive/MyDrive/ime_project/data/porest_device_e_log.csv', names=column_e)

column_plant = ["no", "id", "seq", "air", "monoxide", "dioxide", "pm2d5", "pm10", "temperature", "humidity", "score", "cdate"]
plant = pd.read_csv('/content/drive/MyDrive/ime_project/data/porest_device_plant_log.csv', names=column_plant)

# 장치 정보
devices = pd.read_excel("/content/drive/MyDrive/ime_project/data/porest_devices.xlsx",header=1)

<ipython-input-6-352af6bb5bca>:8: DtypeWarning: Columns (3,5,6,7) have mixed types. Specify dtype option on import or set low_memory=False.
  plant = pd.read_csv('/content/drive/MyDrive/ime_project/data/porest_device_plant_log.csv', names=column_plant)


readme에서 recommend한 IGNORE값 제거

In [ ]:
ahu = ahu[['ahu_no','building','power','inv','occupancy','out_temp','in_temp','zone_min_temp','cdate']]
chiller = chiller[['chiller_no','temp','power','inv','cdate']]
e = e[['id','val','cdate']]
plant = plant[['id','air','monoxide','dioxide','pm2d5','pm10','temperature','humidity','score','cdate']]

In [ ]:
chiller = pd.read_csv("/content/drive/MyDrive/ime_project/data/porest_area_chiller_automatic.csv")
set(chiller['chiller_no'])

{1, 2}

In [ ]:
ahu = pd.read_csv("/content/drive/MyDrive/ime_project/data/porest_area_ahu_automatic.csv")
set(ahu['ahu_no'])

{1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 19, 20, 21, 22, 23, 24}

In [ ]:
ahu.head(10)

,ahu_no,building,power,inv,occupancy,out_temp,in_temp,zone_min_temp,cdate
0,1,A,1,30,8,24.0,\N,\N,2023-01-31 10:09
1,2,A,0,0,8,24.0,\N,\N,2023-01-31 10:09
2,3,A,1,30,8,24.0,\N,\N,2023-01-31 10:09
3,4,A,0,0,5,24.0,\N,\N,2023-01-31 10:09
4,5,A,1,30,5,24.0,\N,\N,2023-01-31 10:09
5,6,A,0,0,3,24.0,\N,\N,2023-01-31 10:09
6,7,A,1,30,3,24.0,\N,\N,2023-01-31 10:09
7,8,A,1,30,5,24.0,\N,\N,2023-01-31 10:09
8,9,A,1,30,5,24.0,\N,\N,2023-01-31 10:09
9,10,A,0,0,7,24.0,\N,\N,2023-01-31 10:09


# ahu_man

**ahu_man**

총 8개의 column으로 구성
- 첫번째 : 단순 numbering이므로 => 제거
- 두번째 : 항상 1이므로 => 제거
- 세번째 : "locate" 주방배기=kitchen , 공조기=ahu => 주방배기의 경우 제거 후 column 제거
- 네번째 : "type" 수동제어=0 , 강제제어or스케줄제어=1
- 다섯번째 : "text" 제어 내용을 줄글로 명시
- 여섯번째 : inverter 정보이므로 => 제거
- 일곱번째 : "cdate_0" 수동제어시 cdate
- 여덟번째 : "cdate_1" 강제제어or스케줄제어시 cdate

**ahu_man['text']**

ex) "[공조기1]스케줄제어::전원:1\"
- [ ]에는 디바이스+디바이스id 꼴로 들어가 있다. 공조기말고도 열원설비도 있는데 공조기만 남기고 없앤다.
- id 별로 dataframe 나누고 디바이스 column 없앤다.

In [ ]:
ahu_man = pd.read_csv("/content/drive/MyDrive/ime_project/data/porest_area_log.csv", names=column_ahu_man)
ahu_man.head()

,numbering,1,locate,type,text,inverter,cdate_0,cdate_1
0,4117,1,kitchen,1,[주방배기1]스케줄제어::전원:1\,인버터:40,NaN,2023-01-01 06:30:04
1,4118,1,kitchen,1,[주방배기2]스케줄제어::전원:1\,인버터:40,NaN,2023-01-01 06:30:04
2,4119,1,kitchen,1,[주방배기3]스케줄제어::전원:1\,인버터:45,NaN,2023-01-01 07:00:03
3,4120,1,ahu,1,[공조기1]스케줄제어::전원:1\,인버터:35,NaN,2023-01-01 08:00:03
4,4121,1,ahu,1,[공조기3]스케줄제어::전원:1\,인버터:35,NaN,2023-01-01 08:00:03


In [ ]:
column_ahu_man = ['numbering','1','locate','type','text','inverter','cdate_0','cdate_1']
ahu_man = pd.read_csv("/content/drive/MyDrive/ime_project/data/porest_area_log.csv", names=column_ahu_man)
ahu_man = ahu_man[ahu_man['locate'] != 'kitchen']
ahu_man = ahu_man[['type','text','cdate_0','cdate_1']]

In [ ]:
import re

ahu_man['device'] = ahu_man['text'].apply(lambda x : re.findall(r'[ㄱ-ㅎ가-힣]+|\d+', (x.split(']')[0].split('[')[-1]))[0])
ahu_man['id'] = ahu_man['text'].apply(lambda x : re.findall(r'[ㄱ-ㅎ가-힣]+|\d+', (x.split(']')[0].split('[')[-1]))[1])
ahu_man = ahu_man[ahu_man['device'] == '공조기']
ahu_man.drop(columns=['device'], inplace=True)

<ipython-input-50-bd4ba63222b5>:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  ahu_man.drop(columns=['device'], inplace=True)


In [ ]:
ahu_man

,type,text,cdate_0,cdate_1,id
3,1,[공조기1]스케줄제어::전원:1\,NaN,2023-01-01 08:00:03,1
4,1,[공조기3]스케줄제어::전원:1\,NaN,2023-01-01 08:00:03,3
5,1,[공조기18]스케줄제어::전원:1\,NaN,2023-01-01 08:00:03,18
6,1,[공조기23]스케줄제어::전원:1\,NaN,2023-01-01 08:00:03,23
7,1,[공조기2]스케줄제어::전원:1\,NaN,2023-01-01 08:50:04,2
...,...,...,...,...,...
22547,0,[공조기5]자동제어변경::1,2023-08-01 12:18:12,NaN,5
22548,0,[공조기7]자동제어변경::1,2023-08-01 12:18:13,NaN,7
22549,0,[공조기8]자동제어변경::1,2023-08-01 12:18:14,NaN,8
22550,0,[공조기9]자동제어변경::1,2023-08-01 12:18:14,NaN,9


In [ ]:
dfs = [group for _, group in ahu_man.groupby('id')]

ahu_man_dct = {}
for i, df_group in enumerate(dfs):
    df_group['cdate_0'] = pd.to_datetime(df_group['cdate_0'])
    df_group['cdate_1'] = pd.to_datetime(df_group['cdate_1'])
    try:
      df_group['cdate_0'] = df_group['cdate_0'].dt.round('10min')
      df_group['cdate_1'] = df_group['cdate_1'].dt.round('10min')
    except: pass
    ahu_man_dct[i+1] = pd.DataFrame(df_group)

In [ ]:
ahu_man_dct[1].info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 822 entries, 3 to 22544
Data columns (total 5 columns):
 #   Column   Non-Null Count  Dtype         
---  ------   --------------  -----         
 0   type     822 non-null    int64         
 1   text     822 non-null    object        
 2   cdate_0  401 non-null    datetime64[ns]
 3   cdate_1  421 non-null    datetime64[ns]
 4   id       822 non-null    object        
dtypes: datetime64[ns](2), int64(1), object(2)
memory usage: 38.5+ KB


In [ ]:
ahu_man_dct[1].head(10)

,type,text,cdate_0,cdate_1,id
3,1,[공조기1]스케줄제어::전원:1\,NaT,2023-01-01 08:00:00,1
30,0,[공조기1]수동제어::인버터: 40Hz,2023-01-01 14:30:00,NaT,1
69,1,[공조기1]스케줄제어::전원:0\,NaT,2023-01-01 20:00:00,1
96,1,[공조기1]스케줄제어::전원:1\,NaT,2023-01-02 08:50:00,1
128,1,[공조기1]스케줄제어::전원:0\,NaT,2023-01-02 20:00:00,1
155,1,[공조기1]스케줄제어::전원:1\,NaT,2023-01-03 08:50:00,1
187,1,[공조기1]스케줄제어::전원:0\,NaT,2023-01-03 20:00:00,1
214,1,[공조기1]스케줄제어::전원:1\,NaT,2023-01-04 08:50:00,1
246,1,[공조기1]스케줄제어::전원:0\,NaT,2023-01-04 20:00:00,1
272,1,[공조기1]스케줄제어::전원:1\,NaT,2023-01-05 08:50:00,1


# ahu

building=A인 row는 제거해야하므로 제거 후 building feature를 날렸다.

ahu_man 처럼 id별로 dataframe 나눴다.

In [ ]:
ahu_a = ahu[ahu['building'] == 'A']
ahu_a.drop(columns=['building'], inplace=True)
ahu_group = ahu_a.groupby('ahu_no')

ahu_dct = {}
for name, group_df in ahu_group:
  ahu_dct[name] = group_df

<ipython-input-15-88ee91766e90>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  ahu_a.drop(columns=['building'], inplace=True)


In [ ]:
ahu_dct[1]

,ahu_no,power,inv,occupancy,out_temp,in_temp,zone_min_temp,cdate
0,1,1,30,8,24.000,\N,\N,2023-01-31 10:09
20,1,1,30,8,24.000,\N,\N,2023-01-31 10:10
40,1,1,30,8,25.000,\N,\N,2023-01-31 10:20
59,1,1,30,8,25.000,\N,\N,2023-01-31 10:30
78,1,1,30,8,25.000,\N,\N,2023-01-31 10:40
...,...,...,...,...,...,...,...,...
28313,1,1,40,6,31.385,26.54,25.25,2023-08-01 15:20
28321,1,1,40,6,31.405,26.63,25.25,2023-08-01 15:30
28329,1,1,40,7,31.315,26.56,25.1,2023-08-01 15:40
28337,1,1,40,7,31.320,26.58,25.1,2023-08-01 15:50


# trial with ahu id=1

In [ ]:
ahuman = ahu_man_dct[1]
ahu = ahu_dct[1]

In [ ]:
ahuman.head()

,type,text,cdate_0,cdate_1,id
3,1,[공조기1]스케줄제어::전원:1\,NaT,2023-01-01 08:00:00,1
30,0,[공조기1]수동제어::인버터: 40Hz,2023-01-01 14:30:00,NaT,1
69,1,[공조기1]스케줄제어::전원:0\,NaT,2023-01-01 20:00:00,1
96,1,[공조기1]스케줄제어::전원:1\,NaT,2023-01-02 08:50:00,1
128,1,[공조기1]스케줄제어::전원:0\,NaT,2023-01-02 20:00:00,1


In [ ]:
# schedule 제어
ahumans = ahuman[ahuman['type']==1]
ahumans.drop(columns=['id'], inplace=True)

# manual 제어
ahumanm = ahuman[ahuman['type']==0]
ahumanm.drop(columns=['id'], inplace=True)

<ipython-input-19-1ff8fbc5c926>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  ahumans.drop(columns=['id'], inplace=True)
<ipython-input-19-1ff8fbc5c926>:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  ahumanm.drop(columns=['id'], inplace=True)


In [ ]:
ahumans.head(20)

,type,text,cdate_0,cdate_1
3,1,[공조기1]스케줄제어::전원:1\,NaT,2023-01-01 08:00:00
69,1,[공조기1]스케줄제어::전원:0\,NaT,2023-01-01 20:00:00
96,1,[공조기1]스케줄제어::전원:1\,NaT,2023-01-02 08:50:00
128,1,[공조기1]스케줄제어::전원:0\,NaT,2023-01-02 20:00:00
155,1,[공조기1]스케줄제어::전원:1\,NaT,2023-01-03 08:50:00
187,1,[공조기1]스케줄제어::전원:0\,NaT,2023-01-03 20:00:00
214,1,[공조기1]스케줄제어::전원:1\,NaT,2023-01-04 08:50:00
246,1,[공조기1]스케줄제어::전원:0\,NaT,2023-01-04 20:00:00
272,1,[공조기1]스케줄제어::전원:1\,NaT,2023-01-05 08:50:00
306,1,[공조기1]스케줄제어::전원:0\,NaT,2023-01-05 20:00:00


In [ ]:
ahumanm.head(20)

,type,text,cdate_0,cdate_1
30,0,[공조기1]수동제어::인버터: 40Hz,2023-01-01 14:30:00,NaT
557,0,[공조기1]수동제어::인버터: 40Hz,2023-01-09 09:00:00,NaT
658,0,[공조기1]수동제어::인버터: 40Hz,2023-01-10 13:40:00,NaT
687,0,[공조기1]수동제어::인버터: 45Hz,2023-01-10 13:50:00,NaT
694,0,[공조기1]수동제어::인버터: 40Hz,2023-01-10 15:30:00,NaT
705,0,[공조기1]수동제어::인버터: 38Hz,2023-01-10 17:10:00,NaT
724,0,[공조기1]수동제어::인버터: 35Hz,2023-01-10 18:10:00,NaT
799,0,[공조기1]수동제어::인버터: 40Hz,2023-01-11 09:10:00,NaT
802,0,[공조기1]수동제어::인버터: 35Hz,2023-01-11 09:20:00,NaT
809,0,[공조기1]수동제어::인버터: 40Hz,2023-01-11 09:20:00,NaT


In [ ]:
# schedule의 on ~ off 이렇게를 그냥 반영하면 될듯 (가설 매일 08:50~20:00)
# out temp , in temp column 채워넣기 >> 이렇게만 해보자, 수동제어는 있다가해

### alog_filtered

In [ ]:
alog = pd.read_csv("/content/drive/MyDrive/ime_project/data/porest_area_log.csv", usecols=[2, 3, 4, 6, 7],
                    names=['type', 'mode', 'text', 'cdate_0', 'cdate_1'], header=None)
alog['text'] = alog['text'].str.replace('::',',').str.replace('[','').str.replace(']',',').str.replace('\\','').str.replace(':',',')
alog = alog[(alog['type'] == 'freezer')|(alog['type'] == 'ahu')].reset_index(drop=True)
alog[['device', 'how_0', 'how_1', 'how_2']] = pd.DataFrame(alog.text.str.rsplit(',', 3).tolist())
alog[['name', 'day']] = pd.DataFrame(alog.device.str.rsplit(',', 1).tolist())
alog['device'] = alog['name'].str.replace(',','')
alog['cdate'] = np.where(pd.isna(alog['cdate_0']), alog['cdate_1'], alog['cdate_0'])
alog['cdate'] = pd.to_datetime(alog['cdate'])
alog['cdate'] = alog['cdate'].dt.round('10min')

alog_filtered = alog[['type', 'mode', 'device', 'how_0', 'how_1', 'how_2', 'day', 'cdate']]

alog_filtered

<ipython-input-53-590ce961bf12>:3: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  alog['text'] = alog['text'].str.replace('::',',').str.replace('[','').str.replace(']',',').str.replace('\\','').str.replace(':',',')
<ipython-input-53-590ce961bf12>:5: FutureWarning: In a future version of pandas all arguments of StringMethods.rsplit except for the argument 'pat' will be keyword-only.
  alog[['device', 'how_0', 'how_1', 'how_2']] = pd.DataFrame(alog.text.str.rsplit(',', 3).tolist())
<ipython-input-53-590ce961bf12>:6: FutureWarning: In a future version of pandas all arguments of StringMethods.rsplit except for the argument 'pat' will be keyword-only.
  alog[['name', 'day']] = pd.DataFrame(alog.device.str.rsplit(',', 1).tolist())


,type,mode,device,how_0,how_1,how_2,day,cdate
0,ahu,1,공조기1,스케줄제어,전원,1,None,2023-01-01 08:00:00
1,ahu,1,공조기3,스케줄제어,전원,1,None,2023-01-01 08:00:00
2,ahu,1,공조기18,스케줄제어,전원,1,None,2023-01-01 08:00:00
3,ahu,1,공조기23,스케줄제어,전원,1,None,2023-01-01 08:00:00
4,ahu,1,공조기2,스케줄제어,전원,1,None,2023-01-01 08:50:00
...,...,...,...,...,...,...,...,...
20831,ahu,0,공조기7,자동제어변경,1,None,None,2023-08-01 12:20:00
20832,ahu,0,공조기8,자동제어변경,1,None,None,2023-08-01 12:20:00
20833,ahu,0,공조기9,자동제어변경,1,None,None,2023-08-01 12:20:00
20834,ahu,0,공조기10,자동제어변경,1,None,None,2023-08-01 12:20:00


In [ ]:
set(alog_filtered['type'])

{'ahu', 'freezer'}

In [ ]:
set(alog_filtered['how_0'])

{'강제제어', '수동제어', '스케줄제어', '자동제어변경'}

In [ ]:
set(alog_filtered['how_1'])

{'0', '1', '인버터', '전원'}

In [ ]:
set(alog_filtered['how_2'])

{' 30Hz',
 ' 31Hz',
 ' 32Hz',
 ' 33Hz',
 ' 34Hz',
 ' 35Hz',
 ' 36Hz',
 ' 37Hz',
 ' 38Hz',
 ' 39Hz',
 ' 40Hz',
 ' 41Hz',
 ' 42Hz',
 ' 43Hz',
 ' 44Hz',
 ' 45Hz',
 ' 46Hz',
 ' 47Hz',
 ' 48Hz',
 ' 49Hz',
 ' 50Hz',
 '0',
 '1',
 None}

In [ ]:
set(alog_filtered['day'])

{None, '금', '목', '수', '월', '일', '토', '화'}

In [ ]:
plant.head()

,id,air,monoxide,dioxide,pm2d5,pm10,temperature,humidity,score,cdate
0,c2-01-d8-a0-1d-58-cf-4c,25,0,400,13,14,25.86,42.61,\N,2023-01-01 00:00:01
1,c2-01-d8-a0-1d-58-65-44,104,0,400,14,15,26.23,42.56,\N,2023-01-01 00:00:09
2,c2-01-d8-a0-1d-58-ce-74,0,1,447,15,17,27.17,41.22,\N,2023-01-01 00:00:18
3,c2-01-d8-a0-1d-59-b4-14,29,0,413,23,26,25.41,46.23,\N,2023-01-01 00:00:19
4,c2-01-d8-a0-1d-58-68-a4,164,0,585,11,12,26.17,36.11,\N,2023-01-01 00:00:26


In [ ]:
e.head()

,id,val,cdate
0,c2-04-d8-a0-1d-58-c7-b8,169.4950,2023-01-01 00:00:00
1,c2-04-d8-a0-1d-58-c7-28,239.4460,2023-01-01 00:00:01
2,c2-04-d8-a0-1d-58-c6-d0,4353.4900,2023-01-01 00:00:03
3,c2-04-d8-a0-1d-58-c6-44,259.6240,2023-01-01 00:00:06
4,c2-04-d8-a0-1d-58-c6-74,49.7724,2023-01-01 00:00:06
